✋ The goal is to build a machine learning model that can automatically assess the relevance of Arabic text documents to a specific topic, assigning a score between 0 and 10.


*  **Steps: ✈**


1.   Data Collection and Preparation:

Choose a Topic and Websites: Select a relevant topic (e.g., "technology," "LLMs") and find Arabic websites covering that topic.

Web Scraping: Use libraries like Beautiful Soup (requests and BeautifulSoup in Python) to extract text content from articles on those websites.

Scoring: Create a dataset by assigning a relevance score (0-10) to each scraped text article using Embedding and TF-IDF techniques.

2. Arabic Text Preprocessing:

  * Segmentation: Split the Arabic text into individual words using a tool like
  Farasa (FarasaSegmenter).

  * Stop Word Removal: Eliminate common Arabic words (like prepositions and conjunctions) that don't carry much meaning using nltk.corpus.stopwords.

  * Stemming: Reduce words to their root form using either light stemming (ArabicLightStemmer from tashaphyne.stemming) or the ISRI Stemmer (ISRIStemmer from nltk.stem.isri).

  * Lemmatization (optional): If possible, convert words to their dictionary form (lemma) using an Arabic lemmatizer. This can improve accuracy but might be more complex.

3. Data Splitting:

  * Divide the dataset into training, validation, and test sets using train_test_split from sklearn.model_selection.

4. Text Vectorization (Numerical Representation):
  * Tokenization: Convert words into numerical representations using Tokenizer from tensorflow.keras.preprocessing.text.
  * Padding: Ensure all text sequences have the same length using pad_sequences from tensorflow.keras.preprocessing.sequence.

5. Building and Training RNN Models:

   Model Architectures: Experiment with different Recurrent Neural Network (RNN) architectures using Keras:
  * Simple RNN (SimpleRNN)
  * Bidirectional RNN (Bidirectional(LSTM) or Bidirectional(GRU))
  * GRU (GRU)
  * LSTM (LSTM)

  Model Compilation: Compile the models using an appropriate loss function (like Mean Squared Error for regression), an optimizer (e.g., adam), and evaluation metrics (e.g., Mean Absolute Error).

  Model Training: Train the models on prepared data and validate their performance using the validation set.

6. Model Evaluation:

  Evaluation Metrics: Evaluate the trained models on the test set using metrics such as:

  * Mean Squared Error (MSE)
  * R-squared (R²)

  ▶▶▶
  


In [99]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem.isri import ISRIStemmer
from qalsadi.lemmatizer import Lemmatizer
import nltk
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import torch
from transformers import BertTokenizer, BertModel
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, GRU, Bidirectional, Dense, SimpleRNN, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import keras_tuner as kt

In [84]:
df = pd.read_csv('/content/drive/Othercomputers/My Laptop/atelier4/ArabicData/ArabicData/scraped_data.csv' , header=None)
df.columns = ['Text']
df.head()

,Text
0,وهو سِمة العصر الحديث البارزة، طوَّره الإنسان ...
1,ورافقت هذا الإنجاز تحدياتٌ اقتصادية سببت فجوة ...
2,يعود طرح مصطلح الذكاء الاصطناعي إلى عالم الحاس...
3,وتم الإعلان في ذلك المؤتمر عن حملات لجمع تبرعا...
4,يعرَّف الذكاء الاصطناعي بأنه البناء المنطقي لل...


# **CLEAN DATA ⌛**


In [85]:
# nltk.download('stopwords')
df['Text_cleaned'] = df['Text'].str.replace(r'[^\u0600-\u06FF\s]+' ,'', regex=True)
stop_words = set(stopwords.words('arabic'))
df['Text_cleaned'] = df['Text_cleaned'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))
df.head()

,Text,Text_cleaned
0,وهو سِمة العصر الحديث البارزة، طوَّره الإنسان ...,سِمة العصر الحديث البارزة، طوَّره الإنسان لخدم...
1,ورافقت هذا الإنجاز تحدياتٌ اقتصادية سببت فجوة ...,ورافقت الإنجاز تحدياتٌ اقتصادية سببت فجوة هائل...
2,يعود طرح مصطلح الذكاء الاصطناعي إلى عالم الحاس...,يعود طرح مصطلح الذكاء الاصطناعي عالم الحاسوب ا...
3,وتم الإعلان في ذلك المؤتمر عن حملات لجمع تبرعا...,وتم الإعلان المؤتمر حملات لجمع تبرعات مالية لد...
4,يعرَّف الذكاء الاصطناعي بأنه البناء المنطقي لل...,يعرَّف الذكاء الاصطناعي بأنه البناء المنطقي لل...


# **ADD SCORE TO DATA [ TF-IDF vs Embedding ] ✅**

In [86]:
topic = 'نماذج اللغة الكبيرة'

# all = [topic] + [''.join(text) for i , text in df['lemm Text'].items() if text is not None]
all = [topic] + df['Text_cleaned'].tolist()
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(all)
# print(tfidf_matrix[0])
# print(tfidf_matrix[1:])
cosine_sim = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1:]).flatten()
for i , score in enumerate(cosine_sim):
  print(i , score)

0 0.0
1 0.0
2 0.0
3 0.0
4 0.0
5 0.0
6 0.0
7 0.0
8 0.0
9 0.0
10 0.0
11 0.0
12 0.0
13 0.0
14 0.05508723802115009
15 0.0
16 0.0
17 0.0
18 0.0
19 0.0
20 0.0
21 0.0
22 0.0
23 0.0
24 0.0
25 0.0
26 0.0
27 0.0
28 0.0
29 0.0
30 0.0
31 0.0
32 0.0
33 0.0
34 0.0
35 0.0
36 0.0
37 0.0
38 0.0
39 0.0
40 0.0
41 0.0
42 0.0
43 0.0
44 0.0
45 0.0
46 0.0
47 0.0
48 0.9431740424119781
49 0.0
50 0.0
51 0.1834685514421972
52 0.1644030007558816
53 0.055236666907747395
54 0.0
55 0.30764950674362074
56 0.22727044529210372
57 0.0
58 0.24138373168554936
59 0.08782663677943245
60 0.0439541585565201
61 0.14039592309136703
62 0.0
63 0.16415099117456342
64 0.067173577977213
65 0.1036387319872804
66 0.0
67 0.0
68 0.032906613921694555
69 0.0
70 0.12893304601030145
71 0.15461739297687613
72 0.30976516710990076
73 0.10507561148358432
74 0.20948452613410434
75 0.17320949463777252
76 0.13630070043395776
77 0.21151368932745093
78 0.0
79 0.30860657558878546
80 0.0662277959150264
81 0.27522383874104245
82 0.0
83 0.0
84 0.0
85 0.

In [87]:
topic = 'نماذج اللغة الكبيرة'

texts = df['Text_cleaned'].tolist()

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

def get_embedding(text , tokenizer):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).detach().numpy()

topic_embedding = get_embedding(topic , tokenizer)
text_embeddings = np.array([get_embedding(text , tokenizer) for text in texts]).squeeze(1)

relevance_scores = cosine_similarity(topic_embedding, text_embeddings).flatten()
scores = []
for i, score in enumerate(relevance_scores):
    print(f"Text {i+1} relevance score: {score}")
    scores.append(f"{score*10:.1f}")
df['relevance_score'] = scores
df['relevance_score'] = pd.to_numeric(df['relevance_score'])
df.head()

Text 1 relevance score: 0.9334957599639893
Text 2 relevance score: 0.9207879304885864
Text 3 relevance score: 0.9156992435455322
Text 4 relevance score: 0.9175287485122681
Text 5 relevance score: 0.9021520614624023
Text 6 relevance score: 0.9179196953773499
Text 7 relevance score: 0.9057813882827759
Text 8 relevance score: 0.9041684865951538
Text 9 relevance score: 0.9130188226699829
Text 10 relevance score: 0.9090552926063538
Text 11 relevance score: 0.9015377759933472
Text 12 relevance score: 0.9107174873352051
Text 13 relevance score: 0.9180015325546265
Text 14 relevance score: 0.9215573072433472
Text 15 relevance score: 0.8910301923751831
Text 16 relevance score: 0.9118050336837769
Text 17 relevance score: 0.9245760440826416
Text 18 relevance score: 0.9134645462036133
Text 19 relevance score: 0.8923670053482056
Text 20 relevance score: 0.9115116596221924
Text 21 relevance score: 0.9219315648078918
Text 22 relevance score: 0.9218125343322754
Text 23 relevance score: 0.88394069671630

,Text,Text_cleaned,relevance_score
0,وهو سِمة العصر الحديث البارزة، طوَّره الإنسان ...,سِمة العصر الحديث البارزة، طوَّره الإنسان لخدم...,9.3
1,ورافقت هذا الإنجاز تحدياتٌ اقتصادية سببت فجوة ...,ورافقت الإنجاز تحدياتٌ اقتصادية سببت فجوة هائل...,9.2
2,يعود طرح مصطلح الذكاء الاصطناعي إلى عالم الحاس...,يعود طرح مصطلح الذكاء الاصطناعي عالم الحاسوب ا...,9.2
3,وتم الإعلان في ذلك المؤتمر عن حملات لجمع تبرعا...,وتم الإعلان المؤتمر حملات لجمع تبرعات مالية لد...,9.2
4,يعرَّف الذكاء الاصطناعي بأنه البناء المنطقي لل...,يعرَّف الذكاء الاصطناعي بأنه البناء المنطقي لل...,9.0


# **PRE-PROCESS DATA**

In [88]:
df['Text tokens'] = df['Text_cleaned'].apply(word_tokenize)
df.head()

,Text,Text_cleaned,relevance_score,Text tokens
0,وهو سِمة العصر الحديث البارزة، طوَّره الإنسان ...,سِمة العصر الحديث البارزة، طوَّره الإنسان لخدم...,9.3,"[سِمة, العصر, الحديث, البارزة،, طوَّره, الإنسا..."
1,ورافقت هذا الإنجاز تحدياتٌ اقتصادية سببت فجوة ...,ورافقت الإنجاز تحدياتٌ اقتصادية سببت فجوة هائل...,9.2,"[ورافقت, الإنجاز, تحدياتٌ, اقتصادية, سببت, فجو..."
2,يعود طرح مصطلح الذكاء الاصطناعي إلى عالم الحاس...,يعود طرح مصطلح الذكاء الاصطناعي عالم الحاسوب ا...,9.2,"[يعود, طرح, مصطلح, الذكاء, الاصطناعي, عالم, ال..."
3,وتم الإعلان في ذلك المؤتمر عن حملات لجمع تبرعا...,وتم الإعلان المؤتمر حملات لجمع تبرعات مالية لد...,9.2,"[وتم, الإعلان, المؤتمر, حملات, لجمع, تبرعات, م..."
4,يعرَّف الذكاء الاصطناعي بأنه البناء المنطقي لل...,يعرَّف الذكاء الاصطناعي بأنه البناء المنطقي لل...,9.0,"[يعرَّف, الذكاء, الاصطناعي, بأنه, البناء, المن..."


In [89]:
stemmer = ISRIStemmer()
stemm  = []
for i in range(len(df['Text tokens'])):
  stemm.append([stemmer.stem(word) for word in df['Text tokens'][i]])
df['stem Text'] = stemm
df.head(2)

,Text,Text_cleaned,relevance_score,Text tokens,stem Text
0,وهو سِمة العصر الحديث البارزة، طوَّره الإنسان ...,سِمة العصر الحديث البارزة، طوَّره الإنسان لخدم...,9.3,"[سِمة, العصر, الحديث, البارزة،, طوَّره, الإنسا...","[سمة, عصر, حدث, رزة،, طور, انس, خدم, جاز, همت,..."
1,ورافقت هذا الإنجاز تحدياتٌ اقتصادية سببت فجوة ...,ورافقت الإنجاز تحدياتٌ اقتصادية سببت فجوة هائل...,9.2,"[ورافقت, الإنجاز, تحدياتٌ, اقتصادية, سببت, فجو...","[رفق, جاز, تحد, اقتصادية, سبب, فجة, هئل, وزع, ..."


In [90]:
lemmatiser = Lemmatizer()
lemm  = []
for i in range(len(df['Text tokens'])):
  lemm.append([lemmatiser.lemmatize(word) for word in df['Text tokens'][i]])
df['lemm Text'] = lemm
df.head(2)

Exception ignored in: <function ArabicDictionary.__del__ at 0x7a3e4c8c2320>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/arramooz/arabicdictionary.py", line 109, in __del__
    self.db_connect.close()
sqlite3.ProgrammingError: SQLite objects created in a thread can only be used in that same thread. The object was created in thread id 134409414848512 and this is thread id 134408384280128.
Exception ignored in: <function ArabicDictionary.__del__ at 0x7a3e4c8c2320>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/arramooz/arabicdictionary.py", line 109, in __del__
    self.db_connect.close()
sqlite3.ProgrammingError: SQLite objects created in a thread can only be used in that same thread. The object was created in thread id 134409414848512 and this is thread id 134408384280128.


,Text,Text_cleaned,relevance_score,Text tokens,stem Text,lemm Text
0,وهو سِمة العصر الحديث البارزة، طوَّره الإنسان ...,سِمة العصر الحديث البارزة، طوَّره الإنسان لخدم...,9.3,"[سِمة, العصر, الحديث, البارزة،, طوَّره, الإنسا...","[سمة, عصر, حدث, رزة،, طور, انس, خدم, جاز, همت,...","[سمة, عصر, حديث, بارز, طور, إنس, خدم, إنجاز, م..."
1,ورافقت هذا الإنجاز تحدياتٌ اقتصادية سببت فجوة ...,ورافقت الإنجاز تحدياتٌ اقتصادية سببت فجوة هائل...,9.2,"[ورافقت, الإنجاز, تحدياتٌ, اقتصادية, سببت, فجو...","[رفق, جاز, تحد, اقتصادية, سبب, فجة, هئل, وزع, ...","[رافق, إنجاز, تحدي, اقتصاد, سب, جو, هائل, توزي..."


In [91]:
df.to_csv('/content/drive/Othercomputers/My Laptop/atelier4/ArabicData/ArabicData/scraped_data_with_score.csv' , index=False)

In [92]:
df

,Text,Text_cleaned,relevance_score,Text tokens,stem Text,lemm Text
0,وهو سِمة العصر الحديث البارزة، طوَّره الإنسان ...,سِمة العصر الحديث البارزة، طوَّره الإنسان لخدم...,9.3,"[سِمة, العصر, الحديث, البارزة،, طوَّره, الإنسا...","[سمة, عصر, حدث, رزة،, طور, انس, خدم, جاز, همت,...","[سمة, عصر, حديث, بارز, طور, إنس, خدم, إنجاز, م..."
1,ورافقت هذا الإنجاز تحدياتٌ اقتصادية سببت فجوة ...,ورافقت الإنجاز تحدياتٌ اقتصادية سببت فجوة هائل...,9.2,"[ورافقت, الإنجاز, تحدياتٌ, اقتصادية, سببت, فجو...","[رفق, جاز, تحد, اقتصادية, سبب, فجة, هئل, وزع, ...","[رافق, إنجاز, تحدي, اقتصاد, سب, جو, هائل, توزي..."
2,يعود طرح مصطلح الذكاء الاصطناعي إلى عالم الحاس...,يعود طرح مصطلح الذكاء الاصطناعي عالم الحاسوب ا...,9.2,"[يعود, طرح, مصطلح, الذكاء, الاصطناعي, عالم, ال...","[يعد, طرح, صطلح, ذكء, صطناع, علم, حسب, امر, جو...","[عاد, طرح, مصطلح, ذكاء, اصطناع, عالم, حاسوب, ا..."
3,وتم الإعلان في ذلك المؤتمر عن حملات لجمع تبرعا...,وتم الإعلان المؤتمر حملات لجمع تبرعات مالية لد...,9.2,"[وتم, الإعلان, المؤتمر, حملات, لجمع, تبرعات, م...","[وتم, اعل, ؤمر, حمل, جمع, برع, الة, دعم, بحث, ...","[تم, إعلان, مؤتمر, حمل, جمع, تبرع, مال, دعم, أ..."
4,يعرَّف الذكاء الاصطناعي بأنه البناء المنطقي لل...,يعرَّف الذكاء الاصطناعي بأنه البناء المنطقي لل...,9.0,"[يعرَّف, الذكاء, الاصطناعي, بأنه, البناء, المن...","[عرف, ذكء, صطناع, بأن, بنء, طقي, رمج, خوارزم, ...","[عرف, ذكاء, اصطناع, أن, بناء, منطق, برمجة, وال..."
...,...,...,...,...,...,...
149,بالإشارة إلى الخطط للسماح للمستخدمين بتخصيص س...,بالإشارة الخطط للسماح للمستخدمين بتخصيص سلوك,9.3,"[بالإشارة, الخطط, للسماح, للمستخدمين, بتخصيص, ...","[شرة, خطط, سمح, خدم, خصص, سلك]","[إشارة, خطط, سماح, مستخدم, تخصيص, سلوك]"
150,بناءً على قيمهم الخاصة. هناك قلق مشابه بين ال...,بناءً قيمهم الخاصة قلق مشابه المحللين الأمنيين...,8.9,"[بناءً, قيمهم, الخاصة, قلق, مشابه, المحللين, ا...","[بنء, قيم, خصة, قلق, شبه, حلل, امن, حكم, استبد...","[بناء, قيم, خاصة, قلق, مشابه, محلل, الأمنيين, ..."
151,ستقيد ضوابط التصدير لإدارة بايدن في أكتوبر 202...,ستقيد ضوابط التصدير لإدارة بايدن ، المدى القصي...,8.8,"[ستقيد, ضوابط, التصدير, لإدارة, بايدن, ،, المد...","[تقد, ضبط, صدر, إدر, بيد, ،, مدى, قصر, قل،, وص...","[قيد, ضوابط, تصدير, إدارة, بايدن, ،, مدى, قصير..."
152,المعهد الدولي للدراسات الاستراتيجية - IISS,المعهد الدولي للدراسات الاستراتيجية,9.5,"[المعهد, الدولي, للدراسات, الاستراتيجية]","[عهد, دول, درس, استراتيجية]","[معهد, دولي, دراسة, الاستراتيجية]"


# **RNN & BidirectionalRNN & GRU & LSM ⛓**

In [93]:
df['data'] = [' '.join(df['lemm Text'][i]) for i in range(len(df))]

X = df['data']
y = df['relevance_score']

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [94]:
tokenizer = Tokenizer(num_words=5000, oov_token='<OOV>') # Consider top 5000 words
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_val_seq = tokenizer.texts_to_sequences(X_val)
X_test_seq = tokenizer.texts_to_sequences(X_test)

max_length = 100
X_train_pad = pad_sequences(X_train_seq, maxlen=max_length, padding='post', truncating='post')
X_val_pad = pad_sequences(X_val_seq, maxlen=max_length, padding='post', truncating='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=max_length, padding='post', truncating='post')
X_train_pad

array([[ 283,  284,  475, ...,    0,    0,    0],
       [ 287,    5,    6, ...,    0,    0,    0],
       [ 485,   84,  486, ...,    0,    0,    0],
       ...,
       [  55, 1021,   45, ...,    0,    0,    0],
       [  10,   56,   12, ...,    0,    0,    0],
       [  10,  209, 1031, ...,    0,    0,    0]], dtype=int32)

# **MODELS {WITHOUT FINE-TUNNING} ⛔**

In [95]:
# 1. Simple RNN
model_rnn = Sequential()
model_rnn.add(Embedding(5000, 128, input_length=max_length))
model_rnn.add(SimpleRNN(128))
model_rnn.add(Dense(1, activation='linear')) # Regression output

# 2. Bidirectional RNN
model_bi_rnn = Sequential()
model_bi_rnn.add(Embedding(5000, 128, input_length=max_length))
model_bi_rnn.add(Bidirectional(GRU(128)))
model_bi_rnn.add(Dense(1, activation='linear'))

# 3. GRU
model_gru = Sequential()
model_gru.add(Embedding(5000, 128, input_length=max_length))
model_gru.add(GRU(128))
model_gru.add(Dense(1, activation='linear'))

# 4. LSTM
model_lstm = Sequential()
model_lstm.add(Embedding(5000, 128, input_length=max_length))
model_lstm.add(LSTM(128))
model_lstm.add(Dense(1, activation='linear'))


models = [model_rnn, model_bi_rnn, model_gru, model_lstm]
for model in models:
    print(f"{model.name} training ....")
    model.summary()
    model.compile(loss='mse', optimizer='adam', metrics=['mae'])
    model.fit(X_train_pad, y_train, epochs=20, validation_data=(X_val_pad, y_val))
    print(f"Training Done")

sequential_1 training ....
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 100, 128)          640000    
                                                                 
 simple_rnn (SimpleRNN)      (None, 128)               32896     
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 673025 (2.57 MB)
Trainable params: 673025 (2.57 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/20
4/4 [==============================] - 2s 160ms/step - loss: 71.2086 - mae: 8.2069 - val_loss: 51.9118 - val_mae: 7.0502
Epoch 2/20
4/4 [==============================] - 0s 59ms/step - loss: 41.9659 - mae: 6.2825 - val_loss: 24.5024 - val_mae:

# **EVALUATION ✈**


In [96]:
for model in models:
    y_pred = model.predict(X_test_pad)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print("Model:", model.name)
    print("MSE:", mse)
    print("R-squared:", r2)

1/1 [==============================] - 0s 165ms/step
Model: sequential_1
MSE: 0.9662229155577492
R-squared: -0.11782839110079424
1/1 [==============================] - 1s 785ms/step
Model: sequential_2
MSE: 0.7607770824113939
R-squared: 0.11985297768746928
1/1 [==============================] - 0s 443ms/step
Model: sequential_3
MSE: 0.9707679982760041
R-squared: -0.12308662128821868
1/1 [==============================] - 0s 447ms/step
Model: sequential_4
MSE: 1.0266678628434254
R-squared: -0.1877574696670139


# **MODELS {WITH FINE-TUNNING} ✍**

In [103]:
def lstm_tuned(hp):
  model = Sequential()
  model.add(Embedding(5000 , hp.Int("Embedding_di" ,  min_value=16 , max_value=256 , step=32), input_length=max_length))
  model.add(LSTM(units = hp.Int('units' , min_value=16 , max_value=256 , step=32),
                 dropout = hp.Float('dropout' , min_value=0.2 , max_value=0.5 , step = 0.1),
                 recurrent_dropout = hp.Float('rec_drop' , min_value=0.2 , max_value=0.5 , step = 0.1 )))
  model.add(Dense(1 , activation='linear'))
  model.compile(loss='mse', optimizer='adam', metrics=['mae'])
  return model

def bi_lstm_tuned(hp):
  model = Sequential()
  model.add(Embedding(5000 , hp.Int("Embedding_di" ,  min_value=16 , max_value=256 , step=32), input_length=max_length))
  model.add(Bidirectional(LSTM(units = hp.Int('units' , min_value=16 , max_value=256 , step=32),
                dropout = hp.Float('dropout' , min_value=0.2 , max_value=0.5 , step = 0.1),
                recurrent_dropout = hp.Float('rec_drop' , min_value=0.2 , max_value=0.5 , step = 0.1 ))))
  model.add(Dense(1 , activation='linear'))
  model.compile(loss='mse', optimizer='adam', metrics=['mae'])
  return model

def gru_tuned(hp):
  model = Sequential()
  model.add(Embedding(5000 , hp.Int("Embedding_di" ,  min_value=16 , max_value=256 , step=32), input_length=max_length))
  model.add(GRU(units = hp.Int('units' , min_value=16 , max_value=256 , step=32),
                dropout = hp.Float('dropout' , min_value=0.2 , max_value=0.5 , step = 0.1),
                recurrent_dropout = hp.Float('rec_drop' , min_value=0.2 , max_value=0.5 , step = 0.1 )))
  model.add(Dense(1 , activation='linear'))
  model.compile(loss='mse', optimizer='adam', metrics=['mae'])
  return model

def rnn_tuned(hp):
  model = Sequential()
  model.add(Embedding(5000 , hp.Int("Embedding_di" ,  min_value=16 , max_value=256 , step=32), input_length=max_length))
  model.add(SimpleRNN(units = hp.Int('units' , min_value=16 , max_value=256 , step=32),
                dropout = hp.Float('dropout' , min_value=0.2 , max_value=0.5 , step = 0.1),
                recurrent_dropout = hp.Float('rec_drop' , min_value=0.2 , max_value=0.5 , step = 0.1 )))
  model.add(Dense(1 , activation='linear'))
  model.compile(loss='mse', optimizer='adam', metrics=['mae'])
  return model

In [104]:
path = '/content/drive/Othercomputers/My Laptop/atelier4/models/tuned_models'

tuner_lstm_tuned = kt.Hyperband(lstm_tuned, objective='val_mae' , max_epochs=10 , factor = 3 ,directory=path , project_name='tuner_lstm_tuned')
tuner_lstm_tuned.search(X_train_pad , y_train , epochs=10 , validation_data=(X_val_pad , y_val))

tuner_bi_lstm_tuned = kt.Hyperband(bi_lstm_tuned, objective='val_mae' , max_epochs=10 , factor = 3 ,directory=path , project_name='tuner_bi_lstm_tuned')
tuner_bi_lstm_tuned.search(X_train_pad , y_train , epochs=10 , validation_data=(X_val_pad , y_val))

tuner_gru_tuned = kt.Hyperband(gru_tuned, objective='val_mae' , max_epochs=10 , factor = 3 ,directory=path, project_name='tuner_gru_tuned')
tuner_gru_tuned.search(X_train_pad , y_train , epochs=10 , validation_data=(X_val_pad , y_val))

tuner_rnn_tuned = kt.Hyperband(rnn_tuned, objective='val_mae' , max_epochs=10 , factor = 3 ,directory=path, project_name='tuner_rnn_tuned')
tuner_rnn_tuned.search(X_train_pad , y_train , epochs=10 , validation_data=(X_val_pad , y_val))

models_tuned = []
best_model_lstm_tuned = tuner_lstm_tuned.get_best_models(num_models=1)[0]
models_tuned.append(best_model_lstm_tuned)

best_model_gru_tuned = tuner_gru_tuned.get_best_models(num_models=1)[0]
models_tuned.append(best_model_gru_tuned)

best_model_rnn_tuned = tuner_rnn_tuned.get_best_models(num_models=1)[0]
models_tuned.append(best_model_rnn_tuned)

best_model_bi_lstm_tuned = tuner_bi_lstm_tuned.get_best_models(num_models=1)[0]
models_tuned.append(best_model_bi_lstm_tuned)


Trial 30 Complete [00h 00m 07s]
val_mae: 6.962698459625244

Best val_mae So Far: 0.5809074640274048
Total elapsed time: 00h 03m 01s


In [105]:
for model in models_tuned:
  print(f"{model.name} training ....")
  y_pred = model.predict(X_test_pad)
  mse = mean_squared_error(y_test, y_pred)
  mae = mean_absolute_error(y_test, y_pred)
  r2 = r2_score(y_test, y_pred)
  print("MSE:", mse)
  print("mae:", mae)
  print("R-squared:", r2)

sequential training ....
1/1 [==============================] - 0s 345ms/step
MSE: 0.8748876634836872
mae: 0.3362313906351724
R-squared: -0.012162155874113623
sequential training ....
1/1 [==============================] - 0s 391ms/step
MSE: 0.8902282395457467
mae: 0.32736736933390304
R-squared: -0.029909749293705268
sequential training ....
1/1 [==============================] - 0s 208ms/step
MSE: 0.9244329131334235
mae: 0.34169467290242544
R-squared: -0.06948131671256497
sequential training ....
1/1 [==============================] - 1s 669ms/step
MSE: 0.876785760154597
mae: 0.32339352766672774
R-squared: -0.014358073931565407
